# core

> Read and query chronicle parquet files.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import polars as pl
import pyarrow.parquet as pq
import pyarrow.dataset as ds
from s3fs import S3FileSystem
import pandas as pd
import plotly.express as px
from fastcore.basics import patch

## Read chronicle parquet files

Chronicle collects and stores logs and metrics in a series of parquet files.

Use `read_chronicle()` to read either logs or metrics, by specifying the path to the parquet set you need.

The file tree looks like this, with `logs` and `metrics` in separate folders inside `v1`.

``` bash
.
└── v1/
    ├── logs/
    └── metrics/
```

Inside both `logs` and `metrics` the data is stored by date, separated by year, month and day.

``` bash
.
└── v1/
    ├── logs/
    │   └── 2023/
    │       ├── 02/
    │       │   ├── 01
    │       │   ├── 02
    │       │   ├── 03
    │       │   ├── 04
    │       │   ├── 05
    │       │   └── ...
    │       ├── 03
    │       ├── 04
    │       └── ...
    └── metrics/
        └── 2023/
            ├── 02/
            │   ├── 01
            │   ├── 02
            │   ├── 03
            │   ├── 04
            │   ├── 05
            │   └── ...
            ├── 03
            ├── 04
            └── ...
```

In [ ]:
#| export
def read_chronicle(
        path: str # Path to dataset
    ) -> pl.DataFrame:
    "Read a chronicle parquet file into a polars dataframe."
    return pl.from_arrow(pq.read_table(path))


In [ ]:
z = read_chronicle("./data/v1/metrics")
assert type(z) == pl.dataframe.frame.DataFrame

z = read_chronicle("./data/v1/logs")
assert type(z) == pl.dataframe.frame.DataFrame

## Analyse metrics

In [ ]:
#| export
@pl.api.register_dataframe_namespace("metrics")
class ChronicleMetrics:
    def __init__(self, df: pl.DataFrame):
        self._df = df



C:\Users\apdev\AppData\Local\Temp\ipykernel_19088\3067508964.py:3: UserWarning:

Overriding existing custom namespace 'metrics' (on DataFrame)



In [ ]:
#| export
@patch
def describe(self: ChronicleMetrics) -> pl.DataFrame:
    "Reads metrics dataframe and returns a pandas dataframe with summary of service, name and description of all metrics"
    return (
        self._df
        .groupby("service", "name")
        .agg(
            pl.col("description").unique(),
            pl.col("value_column").unique(),
        )
        .sort("service", "name")
        .to_pandas()
    )


The metrics data has a single row for each collected metric.

Use `describe()` to get a DataFrame of the unique metrics in the metrics data, containing the `service`, `name` and `description` of each metric.

In [ ]:

read_chronicle("./data/v1/metrics/").metrics.describe()

,service,name,description,value_column
0,connect-metrics,rsconnect_system_memory_available,[Graphite metric rsconnect_system_memory_avail...,[value_float]
1,connect-metrics,rsconnect_system_memory_total,[Graphite metric rsconnect_system_memory_total],[value_float]
2,connect-metrics,rsconnect_system_memory_used,[Graphite metric rsconnect_system_memory_used],[value_float]
3,workbench-metrics,rstudio_system_memory_available,[Graphite metric rstudio_system_memory_available],[value_float]
4,workbench-metrics,rstudio_system_memory_total,[Graphite metric rstudio_system_memory_total],[value_float]
5,workbench-metrics,rstudio_system_memory_used,[Graphite metric rstudio_system_memory_used],[value_float]


In [ ]:
#| export
@patch
def filter(self: ChronicleMetrics, 
        name:str, # name of metric to extract
        alias:str = None # alias to use for new column
    ) -> pd.DataFrame:
    "Extract a single metric from a metrics dataframe"
    if alias == None:
        alias = name
    return (
        self._df
        .lazy()
        .filter(pl.col("name") == name)
        .sort(pl.col("host"), pl.col("timestamp"))
        .select([
            "host",
            pl.col("timestamp"),
            pl.col("value_float").alias(alias)
        ])
        .collect()
        .to_pandas()
    )


The `name` argument is used to filter the DataFrame on the `name` column.

In [ ]:
m = read_chronicle("./data/v1/metrics/").metrics.filter("rsconnect_system_memory_used")
assert type(m) == pd.DataFrame
assert list(m) == ['host', 'timestamp', 'rsconnect_system_memory_used']

m = read_chronicle("./data/v1/metrics/").metrics.filter("rsconnect_system_memory_used", "memory")
assert type(m) == pd.DataFrame
assert list(m) == ['host', 'timestamp', 'memory']

m


,host,timestamp,memory
0,rstudio-connect-68785f94cc-qzvrm,2023-04-03 16:00:29.980,1.170264e+09
1,rstudio-connect-68785f94cc-qzvrm,2023-04-03 16:01:29.980,1.187889e+09
2,rstudio-connect-68785f94cc-qzvrm,2023-04-03 16:02:29.980,1.188659e+09
3,rstudio-connect-68785f94cc-qzvrm,2023-04-03 16:03:29.980,1.252348e+09
4,rstudio-connect-68785f94cc-qzvrm,2023-04-03 16:04:29.980,1.259856e+09
...,...,...,...
475,rstudio-connect-68785f94cc-qzvrm,2023-04-03 23:55:29.980,7.842284e+08
476,rstudio-connect-68785f94cc-qzvrm,2023-04-03 23:56:29.980,7.842406e+08
477,rstudio-connect-68785f94cc-qzvrm,2023-04-03 23:57:29.980,7.842406e+08
478,rstudio-connect-68785f94cc-qzvrm,2023-04-03 23:58:29.980,7.882834e+08


In [ ]:
#| export
@patch
def plot(
        self:ChronicleMetrics, # metrics dataframe
        name:str, # name of metric to extract
        alias:str = None # alias to use for new column
    ) -> px.line: 
    "Plot a selected metric using a Plotly line plot"

    dat = self._df.metrics.filter(name, alias) 
    fig = px.line(dat, x='timestamp', y=alias, line_group="host", color="host")
    return fig

In [ ]:
m = read_chronicle("./data/v1/metrics/")
p = m.metrics.plot("rsconnect_system_memory_used", "memory")
assert str(type(p)) == "<class 'plotly.graph_objs._figure.Figure'>"


read_chronicle("./data/v1/metrics/").metrics.plot("rsconnect_system_memory_used", "memory")

## Analyse logs

In [ ]:

#| export
@pl.api.register_dataframe_namespace("logs")
class ChronicleLogs:
    def __init__(self, 
                 df: pl.DataFrame # A polars data frame
                 ):
        "Initialise a chronicle logs DataFrame"
        self._df = df

C:\Users\apdev\AppData\Local\Temp\ipykernel_19088\1723993363.py:3: UserWarning:

Overriding existing custom namespace 'logs' (on DataFrame)



#### Filter logs on type

You can 

In [ ]:
#| export
@patch
def filter_type(self: ChronicleLogs,
                value: str # Value to extract 
    ) -> pd.DataFrame:
    "Extract all logs where type == value"
    return (
        self._df
        .lazy()
        .with_columns([
            (pl.col("body").str.json_path_match(f"$.{value}").alias(f".{value}")),
            (pl.col("body").str.json_path_match("$.type").alias(".type"))
        ])
        .filter(pl.col(f".{value}").is_not_null())
        .select(["service", "host", "timestamp", f".{value}", ".type", "body"])
        .sort("service", "host", f".{value}", "timestamp")
        .collect()
    )

In [ ]:
logs = read_chronicle("./data/v1/logs").logs.filter_type("username")
assert type(logs) == pl.DataFrame

# assert logs


In [ ]:

read_chronicle("./data/v1/logs").logs.filter_type("username")

service,host,timestamp,.username,.type,body
str,str,datetime[ms],str,str,str
"""workbench""","""rstudio-workbe…",2023-04-03 18:01:26.761,"""james""","""session_exit""","""{""pid"":236,""us…"
"""workbench""","""rstudio-workbe…",2023-04-07 03:46:57.960,"""james""","""auth_login""","""{""pid"":1059,""u…"
"""workbench""","""rstudio-workbe…",2023-04-07 03:51:15.161,"""james""","""session_start""","""{""pid"":236,""us…"
"""workbench""","""rstudio-workbe…",2023-04-07 04:19:18.561,"""james""","""session_exit""","""{""pid"":236,""us…"
"""workbench""","""rstudio-workbe…",2023-04-07 04:19:19.764,"""james""","""session_start""","""{""pid"":883,""us…"
"""workbench""","""rstudio-workbe…",2023-04-07 05:43:02.161,"""james""","""session_exit""","""{""pid"":883,""us…"
"""workbench""","""rstudio-workbe…",2023-04-10 13:14:39.961,"""james""","""session_quit""","""{""pid"":1781,""u…"
"""workbench""","""rstudio-workbe…",2023-04-10 13:15:13.561,"""james""","""session_start""","""{""pid"":236,""us…"
"""workbench""","""rstudio-workbe…",2023-04-10 13:21:53.361,"""james""","""session_start""","""{""pid"":237,""us…"


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()